In [305]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [306]:
df = pd.read_csv('tested.csv')

In [307]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [308]:
df = df.drop('PassengerId', axis=1)

In [309]:
df = df.drop('Name', axis=1)
df = df.drop('Cabin', axis=1)
df = df.drop('Ticket', axis=1)

In [310]:
#We can use normally replace in here because we have just 2 variable.
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})

In [311]:
df['Embarked'].unique()

array(['Q', 'S', 'C'], dtype=object)

In [312]:
#As you see in the code of top we have three different variables, therefore, this means we can not change it with roplace method

#df['Embarked'] = df['Embarked'].replace({'Q':1})



#Therefore, we are using get_dummies algorithm the categorical datatype 
#to make data numerical and storing the returned dataFrame in a new variable df1
df1 = pd.get_dummies(df['Embarked'])


#Then we concatenate the dataframes
df = pd.concat([df, df1], axis=1)


df= df.drop('Embarked', axis=1)

In [313]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,0,3,0,34.5,0,0,7.8292,0,1,0
1,1,3,1,47.0,1,0,7.0000,0,0,1
2,0,2,0,62.0,0,0,9.6875,0,1,0
3,0,3,0,27.0,0,0,8.6625,0,0,1
4,1,3,1,22.0,1,1,12.2875,0,0,1


In [314]:
#Naha! we have null values. With them we can not run our algorithm.
df.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare         True
C           False
Q           False
S           False
dtype: bool

In [315]:
#Because we have just one parameter which is null. I decided to drop them because dropping it is not affect our data that much.
df['Fare'].isnull().sum()

1

In [316]:
df.dropna(subset=['Fare'], inplace=True)

In [317]:
df['Fare'].isnull().sum()

0

In [318]:
#However I can not drop all the 86 data beacuse it will affect our algorithm. Therefore, we must predict the ages of persons
df['Age'].isnull().sum()

86

In [319]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,0,3,0,34.5,0,0,7.8292,0,1,0
1,1,3,1,47.0,1,0,7.0000,0,0,1
2,0,2,0,62.0,0,0,9.6875,0,1,0
3,0,3,0,27.0,0,0,8.6625,0,0,1
4,1,3,1,22.0,1,1,12.2875,0,0,1


In [320]:
#Predicting the missing age values

#To do not ruin my original data I create df1.
df1 = df 


#Creating and the spliting data with null and nonull values.
test_data = df1[df1['Age'].isnull()]

#df1 is our train data. with nonull values.
df1.dropna(inplace=True)

#I created my train data with nonull age values. My algorithm learn with it. 
Xage_train = df1.drop('Age', axis=1)
yage_train = df1['Age']


In [321]:
test_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
10,0,3,0,NaN,0,0,7.8958,0,0,1
22,1,1,1,NaN,0,0,31.6833,0,0,1
29,0,3,0,NaN,2,0,21.6792,1,0,0
33,1,3,1,NaN,1,2,23.4500,0,0,1
36,1,3,1,NaN,0,0,8.0500,0,0,1


In [324]:
#import linearregression and fit it into data
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(Xage_train, yage_train)

#In order to predict null values we drop age column
Xage_test = test_data.drop('Age', axis=1)


#We predictted the age column with our model and write it to our data.
yage_prediction = lr.predict(Xage_test)


test_data['Age'] = yage_prediction

C:\Users\ferit\AppData\Local\Temp\ipykernel_20236\2765192682.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Age'] = yage_prediction


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
10,0,3,0,24.111369,0,0,7.8958,0,0,1
22,1,1,1,38.417371,0,0,31.6833,0,0,1
29,0,3,0,21.157066,2,0,21.6792,1,0,0
33,1,3,1,19.953408,1,2,23.4500,0,0,1
36,1,3,1,22.218398,0,0,8.0500,0,0,1


In [337]:
#We concat our data and here we go!! We have any null data.
dfAgePredicted = pd.concat([test_data, df1], ignore_index=True)
dfAgePredicted.isnull().any().sum()

0

In [348]:
#imports
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report

In [349]:
#Making data ready for model

X = dfAgePredicted.drop('Survived', axis=1)
y = dfAgePredicted['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [357]:
#Training data and making prediction.
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        83
           1       1.00      1.00      1.00        55

    accuracy                           1.00       138
   macro avg       1.00      1.00      1.00       138
weighted avg       1.00      1.00      1.00       138

